In [1]:
import json
import numpy as np
import tensorflow as tf
import collections
from sklearn.cross_validation import train_test_split

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
with open('ctexts.json','r') as fopen:
    ctexts = json.load(fopen)
    
with open('headlines.json','r') as fopen:
    headlines = json.load(fopen)

In [3]:
import tensorflow as tf
import collections
from tensorflow.python.util import nest
from tensorflow.contrib.framework.python.framework import tensor_util
from tensorflow.python.layers.core import Dense

class CopyNetWrapperState(
    collections.namedtuple("CopyNetWrapperState", ("cell_state", "last_ids", "prob_c"))):
    def clone(self, **kwargs):
        def with_same_shape(old, new):
            """Check and set new tensor's shape."""
            if isinstance(old, tf.Tensor) and isinstance(new, tf.Tensor):
                return tensor_util.with_same_shape(old, new)
            return new

        return nest.map_structure(
            with_same_shape,
            self,
            super(CopyNetWrapperState, self)._replace(**kwargs))

class CopyNetWrapper(tf.contrib.rnn.RNNCell):
    '''
    A copynet RNN cell wrapper
    '''
    def __init__(self, cell, source_extend_tokens, max_oovs, encode_output, output_layer, vocab_size, name=None):
        '''
        Args:
            - cell: the decoder cell
            - source_extend_tokens: input tokens with oov word ids
            - max_oovs: max number of oov words in each batch
            - encode_output: the output of encoder cell
            - output_layer: the layer used to map decoder output to vocab distribution
            - vocab_size: this is target vocab size
        '''
        super(CopyNetWrapper, self).__init__(name=name)
        self.cell = cell
        self.source_extend_tokens = source_extend_tokens
        self.encode_output = encode_output
        self.max_oovs = max_oovs
        self.output_layer = output_layer
        self._output_size = vocab_size + max_oovs
        self.copy_layer = Dense(self.cell.output_size,activation=tf.tanh,use_bias=False,name="Copy_Weigth")

    def __call__(self, inputs, state):
        last_ids = state.last_ids
        prob_c = state.prob_c
        cell_state = state.cell_state
        
        '''
            - Selective read
                At first, my implement is based on the paper, which looks like belowing:
                    mask = tf.cast(tf.equal(tf.expand_dims(last_ids,1),self.source_extend_tokens), tf.float32)
                    pt = mask * prob_c
                    pt_sum = tf.reduce_sum(pt, axis=1)
                    pt = tf.where(tf.less(pt_sum, 1e-7), pt, pt / tf.expand_dims(pt_sum, axis=1))
                    selective_read = tf.einsum("ijk,ij->ik",self.encode_output, pt)
                It looks OK for me, but I got NAN loss after one training step. I tried tf.Print to debug, but cannot find 
                the problem. Then I tried tfdbg to detect the source of NAN and found it refer to the tf.where code piece, 
                which is 'pt / tf.expand_dims(pt_sum, axis=1)'. 
                I am not sure why this code will got NANs. Maybe pt and pt_sum are not of the same shape, due to the broadcasting,
                we will got NANs, and these NANs will be used to calculate the gradients, and cause NANs values.
                Take a close look at the paper, we will find that p(y_t,c|.) is same for all the same input tokens. And selective_read
                is sum of p_t * h_t, in which p_t is 1/K *(p(x_t,c|.)), K is sum of p(x_t,c|.). So p_t is equal to:
                p_t = 1 / (number of times x_t shows in input tokens)
        '''
        # get selective read
        # batch * input length
        mask = tf.cast(tf.equal(tf.expand_dims(last_ids,1),self.source_extend_tokens), tf.float32)
        mask_sum = tf.reduce_sum(mask, axis=1)
        mask = tf.where(tf.less(mask_sum, 1e-7), mask, mask / tf.expand_dims(mask_sum, axis=1))
        pt = mask * prob_c
        selective_read = tf.einsum("ijk,ij->ik",self.encode_output, pt)

        inputs = tf.concat([inputs, selective_read], axis=-1)
        outputs, cell_state = self.cell(inputs, cell_state)

        # this is generate mode
        vocab_dist = self.output_layer(outputs)

        # this is copy mode
        # batch * length * output size
        copy_score = self.copy_layer(self.encode_output)
        # batch * length
        copy_score = tf.einsum("ijk,ik->ij",copy_score,outputs)

        vocab_size = tf.shape(vocab_dist)[-1]
        extended_vsize = vocab_size + self.max_oovs 

        batch_size = tf.shape(vocab_dist)[0]
        extra_zeros = tf.zeros((batch_size, self.max_oovs))
        # batch * extend vocab size
        vocab_dists_extended = tf.concat(axis=-1, values=[vocab_dist, extra_zeros])

        # this part is same as that of point generator, but using einsum is much simpler.
        source_mask = tf.one_hot(self.source_extend_tokens,extended_vsize)
        attn_dists_projected = tf.einsum("ijn,ij->in", source_mask, copy_score)

        final_dist = vocab_dists_extended + attn_dists_projected

        # This is greeding search, need to test with beam search
        last_ids = tf.argmax(final_dist, axis=-1, output_type=tf.int32)

        # this is used to calculate p(y_t,c|.)
        # safe softmax
        final_dist_max = tf.expand_dims(tf.reduce_max(final_dist,axis=1), axis=1)
        final_dist_exp = tf.reduce_sum(tf.exp(final_dist - final_dist_max),axis=1)
        p_c = tf.exp(attn_dists_projected - final_dist_max) / tf.expand_dims(final_dist_exp, axis=1)
        p_c = tf.einsum("ijn,in->ij",source_mask,p_c)

        state = CopyNetWrapperState(cell_state=cell_state, last_ids=last_ids, prob_c=p_c)
        return final_dist, state

    @property
    def state_size(self):
        """size(s) of state(s) used by this cell.
            It can be represented by an Integer, a TensorShape or a tuple of Integers
            or TensorShapes.
        """
        return CopyNetWrapperState(cell_state=self.cell.state_size, last_ids=tf.TensorShape([]),
            prob_c = self.source_extend_tokens.shape[-1].value)

    @property
    def output_size(self):
        """Integer or TensorShape: size of outputs produced by this cell."""
        return self._output_size

    def zero_state(self, batch_size, dtype):
        with tf.name_scope(type(self).__name__ + "ZeroState", values=[batch_size]):
            cell_state = self.cell.zero_state(batch_size, dtype)
            last_ids = tf.zeros([batch_size], tf.int32) - 1
            prob_c = tf.zeros([batch_size, tf.shape(self.encode_output)[1]], tf.float32)
            return CopyNetWrapperState(cell_state=cell_state, last_ids=last_ids, prob_c=prob_c)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

def topic_modelling(string, n = 500):
    vectorizer = TfidfVectorizer()
    tf = vectorizer.fit_transform([string])
    tf_features = vectorizer.get_feature_names()
    compose = TruncatedSVD(1).fit(tf)
    return ' '.join([tf_features[i] for i in compose.components_[0].argsort()[: -n - 1 : -1]])

In [5]:
%%time
h, c = [], []
for i in range(len(ctexts)):
    try:
        c.append(ctexts[i])
        h.append(headlines[i])
    except:
        pass

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.54 ms


In [6]:
def build_dataset(words, n_words):
    count = [['PAD', 0], ['GO', 1], ['EOS', 2], ['UNK', 3]]
    count.extend(collections.Counter(words).most_common(n_words))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [7]:
concat_from = ' '.join(c).split()
vocabulary_size_from = len(list(set(concat_from)))
data_from, count_from, dictionary_from, rev_dictionary_from = build_dataset(concat_from, vocabulary_size_from)
print('vocab from size: %d'%(vocabulary_size_from))
print('Most common words', count_from[4:10])
print('Sample data', data_from[:10], [rev_dictionary_from[i] for i in data_from[:10]])

vocab from size: 50942
Most common words [('the', 97951), ('dot', 79175), ('comma', 74246), ('and', 42544), ('to', 41157), ('of', 38549)]
Sample data [4, 11504, 7, 13450, 659, 12, 506, 8180, 11, 4096] ['the', 'daman', 'and', 'diu', 'administration', 'on', 'wednesday', 'withdrew', 'a', 'circular']


In [8]:
concat_to = ' '.join(h).split()
vocabulary_size_to = len(list(set(concat_to)))
data_to, count_to, dictionary_to, rev_dictionary_to = build_dataset(concat_to, vocabulary_size_to)
print('vocab to size: %d'%(vocabulary_size_to))
print('Most common words', count_to[4:10])
print('Sample data', data_to[:10], [rev_dictionary_to[i] for i in data_to[:10]])

vocab to size: 8534
Most common words [('to', 1388), ('in', 1196), ('comma', 876), ('s', 785), ('for', 733), ('of', 596)]
Sample data [2797, 14, 2798, 2799, 656, 2800, 5, 1642, 657, 2087] ['daman', 'and', 'diu', 'revokes', 'mandatory', 'rakshabandhan', 'in', 'offices', 'order', 'malaika']


In [9]:
for i in range(len(h)):
    h[i] = h[i] + ' EOS'
h[0]

'daman and diu revokes mandatory rakshabandhan in offices order EOS'

In [10]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [11]:
def str_idx(corpus, dic, UNK=3):
    X = []
    for i in corpus:
        ints = []
        for k in i.split():
            ints.append(dic.get(k, UNK))
        X.append(ints)
    return X

In [12]:
train_X, test_X, train_Y, test_Y = train_test_split(c, h, test_size = 0.2)

In [13]:
def sent2idx(sent, vocab, UNK=UNK):
    tokens = sent.split()
    oovs = []
    extend_tokens = []
    tokenized = []
    for token in tokens:
        if token not in vocab:
            tokenized.append(UNK)
            if token not in oovs:
                oovs.append(token)
            extend_tokens.append(len(vocab) + oovs.index(token))
        else:
            extend_tokens.append(vocab[token])
            tokenized.append(vocab[token])
    return tokenized, extend_tokens, oovs

def target2idx(sent, oovs, vocab,UNK=UNK):
    tokens = sent.split()
    tokenized = []
    for token in tokens:
        if token not in vocab:
            if token not in oovs:
                tokenized.append(UNK)
            else:
                tokenized.append(len(vocab) + oovs.index(token))
        else:
            tokenized.append(vocab[token])
    return tokenized

In [14]:
class Summarization:
    def __init__(self, size_layer, num_layers, embedded_size, 
                 from_dict_size):
        
        def lstm_cell(reuse=False):
            return tf.nn.rnn_cell.GRUCell(size_layer, reuse=reuse)
        
        def attention(encoder_out, seq_len, reuse=False):
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(num_units = size_layer, 
                                                                    memory = encoder_out,
                                                                    memory_sequence_length = seq_len)
            return tf.contrib.seq2seq.AttentionWrapper(
            cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell(reuse) for _ in range(num_layers)]), 
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        self.source_oov_words = tf.placeholder(tf.int32, shape=[])
        self.source_extend_tokens = tf.placeholder(tf.int32, shape=[None, None])
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        
        condition = tf.less(decoder_input, from_dict_size)
        self.decoder_input = self.Y
        self.decoder_input_length = self.Y_seq_len
        self.predict_count = tf.reduce_sum(self.decoder_input_length)
        
        encoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        decoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        encoder_cells = tf.nn.rnn_cell.MultiRNNCell([lstm_cell() for _ in range(num_layers)])
        self.encoder_out, self.encoder_state = tf.nn.dynamic_rnn(cell = encoder_cells, 
                                                                 inputs = encoder_embedded, 
                                                                 sequence_length = self.X_seq_len,
                                                                 dtype = tf.float32)
        self.decode_initial_state = self.encoder_state[-1]
        initial_state = [self.decode_initial_state for i in range(num_layers)]
        self.initial_state = tuple(initial_state)
        decoder_cell = attention(self.encoder_out, self.X_seq_len)
        dense_layer = tf.layers.Dense(embedded_size)
        
        initial_state = decoder_cell.zero_state(batch_size, tf.float32).clone(cell_state=self.initial_state)
        
        self.decode_cell = CopyNetWrapper(decoder_cell, self.source_extend_tokens, 50, 
                                          self.encoder_out, dense_layer, embedded_size)
        
        dense_layer = tf.layers.Dense(from_dict_size)
        
        self.initial_state = self.decode_cell.zero_state(batch_size,
                                                         tf.float32).clone(cell_state=initial_state)
        
        training_helper = tf.contrib.seq2seq.TrainingHelper(
                inputs = tf.nn.embedding_lookup(decoder_embeddings, decoder_input),
                sequence_length = self.decoder_input_length,
                time_major = False)
        training_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = self.decode_cell,
                helper = training_helper,
                initial_state = self.initial_state,
                output_layer = dense_layer)
        training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = training_decoder,
                impute_finished = True,
                maximum_iterations = tf.reduce_max(self.Y_seq_len))
        
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                embedding = encoder_embeddings,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS)
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = self.decode_cell,
                helper = predicting_helper,
                initial_state = self.initial_state,
                output_layer = dense_layer)
        
        predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = True,
                maximum_iterations = tf.reduce_max(self.X_seq_len))
        
        self.training_logits = training_decoder_output.rnn_output
        self.predicting_ids = predicting_decoder_output.sample_id
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        
        self.optimizer = tf.train.AdamOptimizer().minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [15]:
size_layer = 128
num_layers = 2
embedded_size = 128
batch_size = 8
epoch = 5

In [16]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Summarization(size_layer, num_layers, embedded_size, len(dictionary_from))
sess.run(tf.global_variables_initializer())

In [17]:
from keras.preprocessing.sequence import pad_sequences

def batching(X, Y):
    s_, es_, oovs_, target_ = [], [], [], []
    for x, y in zip(X, Y):
        s,es,oovs = sent2idx(x, dictionary_from)
        target = target2idx(y, oovs,dictionary_from)
        s_.append(s)
        es_.append(es)
        oovs_.append(oovs)
        target_.append(target)
    s_ = pad_sequences(s_,padding='post')
    es_ = pad_sequences(es_,padding='post')
    target_ = pad_sequences(target_,padding='post')
    maxlen = max([len(o) for o in oovs_])
    return s_, es_, target_, maxlen

Using TensorFlow backend.


In [18]:
from tqdm import tqdm
from sklearn.utils import shuffle
import time

for EPOCH in range(10):
    lasttime = time.time()
    total_loss, total_accuracy, total_loss_test, total_accuracy_test = 0, 0, 0, 0
    train_X, train_Y = shuffle(train_X, train_Y)
    test_X, test_Y = shuffle(test_X, test_Y)
    pbar = tqdm(range(0, len(train_X), batch_size), desc='train minibatch loop')
    for k in pbar:
        index = min(k+batch_size,len(train_X))
        batch_x, batch_es, batch_y, maxlen = batching(train_X[k: index],
                                                     train_Y[k: index])
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                 model.source_extend_tokens:batch_es,
                                                model.Y:batch_y,
                                                model.source_oov_words:maxlen})
        total_loss += loss
        total_accuracy += acc
        pbar.set_postfix(cost=loss, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc='test minibatch loop')
    for k in pbar:
        index = min(k+batch_size,len(test_X))
        batch_x, batch_es, batch_y, maxlen = batching(test_X[k: index],
                                                     test_Y[k: index])
        acc, loss = sess.run([model.accuracy, model.cost], 
                                      feed_dict={model.X:batch_x,
                                                 model.source_extend_tokens:batch_es,
                                                model.Y:batch_y,
                                                model.source_oov_words:maxlen})
        total_loss_test += loss
        total_accuracy_test += acc
        pbar.set_postfix(cost=loss, accuracy = acc)
        
    total_loss /= (len(train_X) / batch_size)
    total_accuracy /= (len(train_X) / batch_size)
    total_loss_test /= (len(test_X) / batch_size)
    total_accuracy_test /= (len(test_X) / batch_size)
        
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(EPOCH, total_loss, total_accuracy))
    print('epoch: %d, avg loss test: %f, avg accuracy test: %f'%(EPOCH, total_loss_test, total_accuracy_test))

train minibatch loop:   0%|          | 0/440 [00:00<?, ?it/s]

epoch: 0, avg loss: 8.006333, avg accuracy: 0.093200
epoch: 0, avg loss test: 7.498233, avg accuracy test: 0.111772


train minibatch loop:   0%|          | 0/440 [00:00<?, ?it/s]

epoch: 1, avg loss: 6.790594, avg accuracy: 0.109877
epoch: 1, avg loss test: 7.670779, avg accuracy test: 0.111236


train minibatch loop:   0%|          | 0/440 [00:00<?, ?it/s]

...

epoch: 7, avg loss: 2.457255, avg accuracy: 0.409344
epoch: 7, avg loss test: 11.130594, avg accuracy test: 0.118689


train minibatch loop:   0%|          | 0/440 [00:00<?, ?it/s]

epoch: 8, avg loss: 2.118696, avg accuracy: 0.469681
epoch: 8, avg loss test: 12.793685, avg accuracy test: 0.110155


test minibatch loop: 100%|██████████| 110/110 [00:31<00:00,  3.35it/s, accuracy=0.069, cost=16.2] 

epoch: 9, avg loss: 1.810338, avg accuracy: 0.531653
epoch: 9, avg loss test: 12.994831, avg accuracy test: 0.111562


In [19]:
batch_x, batch_es, batch_y, maxlen = batching(test_X[:1], test_Y[:1])

In [20]:
[rev_dictionary_from[i] for i in sess.run(model.predicting_ids, feed_dict = {model.X: batch_x,
                                             model.Y: batch_y,
                                             model.source_extend_tokens:batch_es,
                                             model.source_oov_words:maxlen})[0]]

['india', 'india', 'in', 'indian', 'in', 'EOS']